In [122]:
import numpy as np
import pandas as pd

In [124]:
import datetime

def _apply_operation_state(x, current_date, open_col, close_col, uncompleted_str):
    '''
    based the relationship between current date and close data
    '''
    # opening date
    open_dt = x[open_col]
    if open_dt == uncompleted_str:
        # open_dt = '9999-12-31'
        open_dt = datetime.datetime(year=9999, month=12, day=30)
    if not isinstance(open_dt, datetime.datetime):
        open_dt = datetime.datetime.strptime(open_dt, '%Y-%m-%d')
    
    # closed date
    close_dt = x[close_col]
    if pd.isna(close_dt):
        # close_dt = '9999-12-31'
        close_dt = datetime.datetime(year=9999, month=12, day=31)
    if not isinstance(close_dt, datetime.datetime):
        close_dt = datetime.datetime.strptime(close_dt, '%Y-%m-%d')
    
    assert close_dt > open_dt, 'The closed date must be greater than the opening date'
    
    if current_date < open_dt:
        state = 'not open'
    elif current_date < close_dt:
        state = 'in operation'
    else:
        state = 'closed'
        
    return state  
# ===================================================================================================

def extract_in_operation_metro_line(
    data, 
    opening_col = None, closed_col = None, 
    uncompleted_str = 'u/c',
    current_date = 'now'
):
    '''
    extract current opening station
    
    params:
    ---
    uncompleted_str (str) :
        indicate the uncompleted or unopening metro station in the 'opening_col' columns
    current_date (None or str):
        None : now consider the date restriction
        'now' : datetime.datetime.now().strftime("%Y-%m-%d")
        'YYYY-MM-DD' : i.e., '2024-07-19'
    '''
    if current_date is None:
        pass
    elif current_date == 'now':
        current_date = datetime.datetime.now()
    else:
        current_date = datetime.datetime.strptime(current_date, '%Y-%m-%d')
    
    # select based on closing date and current date
    if current_date is None:
        pass
    else:
        if not(closed_col is None):
            data['operation_state'] = data.apply(_apply_operation_state, args=(current_date, opening_col, closed_col, uncompleted_str), axis='columns')
    
    # drop uncompleted columns
    data = data.query('operation_state == "in operation"')
    
    return data
# ======================================================================================
def load_all_station_info(excel_path):
    '''
    the station_id
    '''
    line_info = pd.read_excel(excel_path, sheet_name='line_info')

    rename_dict = {
        'Line Name'     : 'line_name',
        'Sequence'      : 'order',
        'Station Code'  : 'stn_code',
        'Station Name'  : 'stn_name',
        'Opening Date'  : 'open_dt',
        'Closed Date'   : 'close_dt' }
    
    station_li = []
    
    for sheet_name in line_info['sheet_name'].unique().tolist():
        # load station list
        station = pd.read_excel(excel_path, sheet_name=sheet_name) \
                    .rename(columns=rename_dict) \
                    .assign(line_code = sheet_name)
        
        # extract in-operation stations
        station = extract_in_operation_metro_line(
                station,
                opening_col = 'open_dt', closed_col = 'close_dt',
                uncompleted_str = 'u/c',
                current_date = 'now') \
        
        station_li.append(station)

    station = pd.concat(station_li, axis=0, ignore_index=True)
    
    return station

In [126]:
excel_path = r'Singapore/metro/station_line_list/station_list.xlsx'
line_info = pd.read_excel(excel_path, sheet_name='line_info')

station = load_all_station_info(excel_path) \

station_transfer = station \
    .assign(stn_name = lambda x : x['stn_name'].str.strip()) \
    .groupby('stn_name', as_index=False) \
    .agg(stn_code = ('stn_code', lambda x : '-'.join(list(set(x)))), 
         line_number = ('stn_code', 'count')) \
    .query('line_number > 1') \
    .drop_duplicates()
    # .merge(station[['order', 'stn_name', 'stn_code', 'line_name', 'line_code']], on='stn_name', how='left') \
    # .sort_values(['line_code', 'order'])

In [127]:
station_transfer.to_csv('transfer_station.csv')